In [1]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data

In [2]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
def mnist_binary(images, labels, binary):
#     binary = [binary[0]-1 , binary[1]-1]
    select = [i for i,l in enumerate(labels) if sum(l[binary])>=1]
    x = images[select]
    y = labels[select][:,binary].astype(int)
    return x, y

In [4]:
numbers = [1, 2, 3, 4, 5]
train_x, train_y = mnist_binary(mnist.train.images, mnist.train.labels, numbers)
test_x, test_y = mnist_binary(mnist.test.images, mnist.test.labels, numbers)

In [5]:
len_numbers = len(numbers)

In [6]:
def variable_summaries(var, name):
  """Attach a lot of summaries to a Tensor."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.scalar_summary('mean/' + name, mean)
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_sum(tf.square(var - mean)))
    tf.scalar_summary('sttdev/' + name, stddev)
    tf.scalar_summary('max/' + name, tf.reduce_max(var))
    tf.scalar_summary('min/' + name, tf.reduce_min(var))
    tf.histogram_summary(name, var)

In [7]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

In [8]:
def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [9]:
  def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    """Reusable code for making a simple neural net layer.
    It does a matrix multiply, bias add, and then uses relu to nonlinearize.
    It also sets up name scoping so that the resultant graph is easy to read,
    and adds a number of summary ops.
    """
    # Adding a name scope ensures logical grouping of the layers in the graph.
    with tf.name_scope(layer_name):
      # This Variable will hold the state of the weights for the layer
      with tf.name_scope('weights'):
        weights = weight_variable([input_dim, output_dim])
        variable_summaries(weights, layer_name + '/weights')
      with tf.name_scope('biases'):
        biases = bias_variable([output_dim])
        variable_summaries(biases, layer_name + '/biases')
      with tf.name_scope('Wx_plus_b'):
        preactivate = tf.matmul(input_tensor, weights) + biases
        tf.histogram_summary(layer_name + '/pre_activations', preactivate)
      activations = act(preactivate, 'activation')
      tf.histogram_summary(layer_name + '/activations', activations)
      return activations

In [10]:
# set variables
with tf.name_scope('input'):
  x = tf.placeholder(tf.float32, [None, 784], name='x-input') # 28*28
  y_ = tf.placeholder(tf.float32, [None, len_numbers], name='y-input')

In [11]:
with tf.name_scope('input_reshape'):
  image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
  tf.image_summary('input', image_shaped_input, 10) # 10 sample in IMAGES of tensorboard 

In [12]:
# Set graphs and GRAPHS of tensorboard
y = nn_layer(x, 784, len_numbers, 'layer1', tf.nn.softmax)

In [13]:
# Set loss function
with tf.name_scope('cross_entropy'):
  diff = y_ * tf.log(y + 1e-10)
  with tf.name_scope('total'):
    cross_entropy = -tf.reduce_mean(diff)
  tf.scalar_summary('cross entropy', cross_entropy)

In [14]:
# # Set hinge loss function for binary class
# with tf.name_scope('cross_entropy'):
#   with tf.name_scope('total'):
#     hinge_loss = tf.reduce_mean(tf.maximum(0., 1. - (y_[:,1] - y_[:,0])*y[:,1]))
#   tf.scalar_summary('hinge_loss', hinge_loss)

In [15]:
# Set hinge loss function for multiclass
with tf.name_scope('cross_entropy'):
  true_index = tf.cast(y_, dtype = bool)
  margin = tf.boolean_mask(y, true_index)
  false_prediction = tf.reshape(tf.boolean_mask(y, ~true_index), [tf.shape(y_)[0], len_numbers-1])
  false_prediction_max = tf.reduce_max(false_prediction, reduction_indices=[1])
  loss = 1 - (margin - false_prediction_max)
  loss = tf.maximum(loss, tf.zeros_like(loss))
  with tf.name_scope('total'):
    hinge_loss = tf.reduce_mean(loss)
  tf.scalar_summary('hinge_loss', hinge_loss)

In [16]:
# Set optimizer method
with tf.variable_scope("trainer") as scope:
    optimizer = tf.train.GradientDescentOptimizer(0.01)
#     trainer = optimizer.minimize(cross_entropy)
    trainer = optimizer.minimize(hinge_loss)

/home/hsuehhungcheng/.pyenv/versions/anaconda-4.0.0/lib/python2.7/site-packages/tensorflow/python/ops/gradients.py:90: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [17]:
with tf.name_scope('accuracy'):
  with tf.name_scope('correct_prediction'):
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
  with tf.name_scope('accuracy'):
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  tf.scalar_summary('accuracy', accuracy)

In [18]:
merged = tf.merge_all_summaries()

In [19]:
sess = tf.Session()

In [20]:
train_writer = tf.train.SummaryWriter('mnist_logs' + '/train', sess.graph)
test_writer = tf.train.SummaryWriter('mnist_logs' + '/test', sess.graph)

In [21]:
sess.run(tf.initialize_all_variables())

In [22]:
for i in range(1000): # train 1000 times
    sess.run(trainer, feed_dict={x: train_x, y_: train_y})
    result = sess.run(merged, feed_dict={x: train_x, y_: train_y})
    train_writer.add_summary(result, i)
    if i % 50 == 0: # save summary if after train 50 times
        result, acc = sess.run([merged, accuracy], feed_dict={x: test_x, y_: test_y})
        test_writer.add_summary(result, i)
        print "accuracy is %0.8s at times %4d" % (acc,i)

accuracy is 0.064739 at times    0
accuracy is 0.195803 at times   50
accuracy is 0.324292 at times  100
accuracy is 0.46565 at times  150
accuracy is 0.599089 at times  200
accuracy is 0.679073 at times  250
accuracy is 0.721639 at times  300
accuracy is 0.752524 at times  350
accuracy is 0.775094 at times  400
accuracy is 0.799644 at times  450
accuracy is 0.823401 at times  500
accuracy is 0.843199 at times  550
accuracy is 0.859434 at times  600
accuracy is 0.868937 at times  650
accuracy is 0.875272 at times  700
accuracy is 0.883191 at times  750
accuracy is 0.889527 at times  800
accuracy is 0.893288 at times  850
accuracy is 0.895268 at times  900
accuracy is 0.897446 at times  950


# Test

In [23]:
print sess.run(y_, feed_dict={x: train_x, y_: train_y})
print sess.run(true_index, feed_dict={x: train_x, y_: train_y})

[[ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 1.  0.  0.  0.  0.]
 ..., 
 [ 1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  1.]]
[[False False  True False False]
 [False False False  True False]
 [ True False False False False]
 ..., 
 [ True False False False False]
 [False False  True False False]
 [False False False False  True]]


In [24]:
sess.run(y_, feed_dict={x: train_x, y_: train_y})

array([[ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.]], dtype=float32)

In [25]:
sess.run(y, feed_dict={x: train_x, y_: train_y})

array([[ 0.01380698,  0.01795236,  0.88862532,  0.00985783,  0.06975739],
       [ 0.0019449 ,  0.00906196,  0.0793579 ,  0.83552766,  0.07410766],
       [ 0.97283369,  0.00863509,  0.01037223,  0.00349912,  0.00465986],
       ..., 
       [ 0.76397556,  0.09182667,  0.07625485,  0.02989758,  0.03804537],
       [ 0.00455048,  0.02500125,  0.91919816,  0.00381381,  0.04743636],
       [ 0.02219394,  0.00259264,  0.05593104,  0.03593724,  0.88334519]], dtype=float32)

In [26]:
sess.run(tf.boolean_mask(y, true_index), feed_dict={x: train_x, y_: train_y})

array([ 0.88862532,  0.83552766,  0.97283369, ...,  0.76397556,
        0.91919816,  0.88334519], dtype=float32)

In [27]:
sess.run(tf.reshape(tf.boolean_mask(y, ~true_index), [tf.shape(y_)[0], len_numbers-1]), feed_dict={x: train_x, y_: train_y})

array([[ 0.01380698,  0.01795236,  0.00985783,  0.06975739],
       [ 0.0019449 ,  0.00906196,  0.0793579 ,  0.07410766],
       [ 0.00863509,  0.01037223,  0.00349912,  0.00465986],
       ..., 
       [ 0.09182667,  0.07625485,  0.02989758,  0.03804537],
       [ 0.00455048,  0.02500125,  0.00381381,  0.04743636],
       [ 0.02219394,  0.00259264,  0.05593104,  0.03593724]], dtype=float32)

In [28]:
sess.run(tf.reduce_max(tf.reshape(tf.boolean_mask(y, ~true_index), [tf.shape(y_)[0], len_numbers-1]), reduction_indices=[1]), 
             feed_dict={x: train_x, y_: train_y})

array([ 0.06975739,  0.0793579 ,  0.01037223, ...,  0.09182667,
        0.04743636,  0.05593104], dtype=float32)

In [29]:
sess.run(tf.boolean_mask(y, true_index) - (tf.reduce_max(tf.reshape(tf.boolean_mask(y, ~true_index), [tf.shape(y_)[0], len_numbers-1]), reduction_indices=[1])), 
             feed_dict={x: train_x, y_: train_y})

array([ 0.81886792,  0.75616974,  0.96246147, ...,  0.67214888,
        0.8717618 ,  0.82741416], dtype=float32)

In [30]:
sess.run(1 - (tf.boolean_mask(y, true_index) - (tf.reduce_max(tf.reshape(tf.boolean_mask(y, ~true_index), [tf.shape(y_)[0], len_numbers-1]), reduction_indices=[1]))), 
             feed_dict={x: train_x, y_: train_y})

array([ 0.18113208,  0.24383026,  0.03753853, ...,  0.32785112,
        0.1282382 ,  0.17258584], dtype=float32)

In [ ]:
train_writer.close()
test_writer.close()

In [ ]:
!pwd

# shell: 
```
tensorboard --logdir=mnist_logs/
```
# Browser: http://127.0.0.1:6006